In [1]:
from openai import OpenAI
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text

In [4]:
api_key = os.environ.get('openai_api_key')

In [5]:
df = pd.read_csv('sales_data_sample.csv')

In [6]:
# create a temp databese in memory
temp_db = create_engine('sqlite:///:memory:', echo=True)


In [7]:
#push the pandas df ---> to temp db for us to be able to interact
data = df.to_sql(name='Sales', con=temp_db)

2024-01-24 15:42:18,204 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 15:42:18,210 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-01-24 15:42:18,210 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-24 15:42:18,211 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-01-24 15:42:18,211 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-24 15:42:18,212 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [8]:
#SQL query on TEMP DB
with temp_db.connect() as conn:
    result = conn.execute(text(
        "SELECT SUM(SALES) FROM Sales GROUP BY QTR_ID LIMIT 5"
        ))

2024-01-24 15:42:19,261 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 15:42:19,262 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales GROUP BY QTR_ID LIMIT 5
2024-01-24 15:42:19,262 INFO sqlalchemy.engine.Engine [generated in 0.00153s] ()
2024-01-24 15:42:19,263 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
result.all()

[(2350817.7300000004,),
 (2048120.2999999986,),
 (1758910.8099999994,),
 (3874780.01,)]

In [10]:
#describe the actual database you are using to gpt; say this is the table, these are the columns
def create_table_definition(df, table_name):
    prompt = f"""### sqlite SQL table, with it properties:
    #
    # {table_name}({','.join(str(col) for col in df.columns)})
    #
    """
    return prompt


In [11]:
print(create_table_definition(df, table_name='Sales'))

### sqlite SQL table, with it properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    


In [15]:
def prompt_input():
    nlp_text = input("What you want from the table? ")
    return nlp_text

In [13]:
def combine_prompts(df,table_name, query_prompt):
    table_definition = create_table_definition(df, table_name)
    query_init_string = f'### A query to answer: {query_prompt}\nSELECT'
    return table_definition+query_init_string

In [14]:
nlp_text = prompt_input() #NLP
combine_prompts(df=df, table_name='Sales', query_prompt=nlp_text) # DF + query that does... + NLP


'### sqlite SQL table, with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: w\nSELECT'

In [16]:
client = OpenAI()



OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable